In [1]:
from langchain.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [2]:
from langchain.utilities import SQLDatabase

In [3]:
db = SQLDatabase.from_uri("sqlite:///./transcriptions.db")

In [4]:
def get_schema(_):
    return db.get_table_info()

In [5]:
def run_query(query):
    return db.run(query)

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

model = ChatOpenAI()

sql_response = (
        RunnablePassthrough.assign(schema=get_schema)
        | prompt
        | model.bind(stop=["\nSQLResult:"])
        | StrOutputParser()
    )

In [7]:
sql_response.invoke({"question": ""})

"SELECT id, date, source_url, transcription, summary, issue_category, contact\nFROM transcriptions\nWHERE summary LIKE '%subscription activation%' OR summary LIKE '%confirming their number%'"

In [8]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [9]:
full_chain = (
    RunnablePassthrough.assign(query=sql_response) 
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response 
    | model
)

In [10]:
ans = full_chain.invoke({"question": ""})
import pprint
from pprint import pprint
pprint(ans)

AIMessage(content='The SQL query "SELECT summary FROM transcriptions WHERE issue_category = \'Other\'" retrieves the summary column from the transcriptions table where the issue_category is \'Other\'. The SQL response provides a list of summaries that match the query. These summaries describe various customer issues and interactions with the customer care representatives. Some examples include customers asking about subscription activation, trouble with internet connection, and accessing internet packages. The responses from the representatives vary depending on the specific issue raised by the customers.')


In [11]:
ans = full_chain.invoke({"question": "do we have angry customers?"})
import pprint
from pprint import pprint
pprint(ans)

AIMessage(content='Based on the given table schema, there are no angry customers in the transcriptions table. The SQL query was executed and the response shows that there are zero occurrences of the word "angry" in the transcription or summary columns.')


In [12]:
ans = full_chain.invoke({"question": "how many?"})
import pprint
from pprint import pprint
pprint(ans)

AIMessage(content='There are 61 rows in the "transcriptions" table.')


In [13]:
#Lets Create a function out of the process
def sqlhandler(sqlfield):
    see = sql_response.invoke({"question": sqlfield})
    print(see)
    template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
    {schema}

    Question: {question}
    SQL Query: {query}
    SQL Response: {response}"""
    prompt_response = ChatPromptTemplate.from_template(template)

    full_chain = (
    RunnablePassthrough.assign(query=sql_response) 
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response 
    | model
    )



In [14]:
sqlfield = "" #Which table or column do you want to focus on
sqlhandler(sqlfield)
#Ask questions based on the mentioned field
ans = full_chain.invoke({"question": ""})
import pprint
from pprint import pprint
pprint(ans)

SELECT id, date, source_url, transcription, summary, issue_category, contact
FROM transcriptions
WHERE id = 2
AIMessage(content='The SQL query SELECTs the values of id, date, source_url, transcription, summary, issue_category, and contact from the transcriptions table WHERE the id is equal to 1. The SQL response returns a single row with the id, date, source URL, transcription, summary, issue category, and contact information for the transcription with an id of 1. The transcription text includes the customer\'s question about their subscription activation and confirming their number, and the customer care representative\'s response confirming the WhatsApp payment and asking for the customer\'s number. The issue category is marked as "Other" and there is no specific contact information provided.')
